In [16]:
# H1) 설정 & 매니페스트 로드
import json
from pathlib import Path

CSR_DIR = Path("/home/yyb02274/yolov5/csr_dump_ratio[0.9]")
WS_DIR = Path("/home/yyb02274/yolov5/ws_dump_ratio[0.9]")
HUF_DIR = Path("/home/yyb02274/yolov5/Deep_Compression_ratio[0.9]/huff_dump_ratio[0.9]")
HUF_DIR.mkdir(exist_ok=True)

# (옵션) row_ptr도 델타 허프만 압축할지 여부
COMPRESS_ROW_PTR = True

mani   = json.loads((CSR_DIR / "manifest_ratio[0.9].json").read_text())
mani_ws= json.loads((WS_DIR / "manifest_ws.json").read_text())
print("layers:", len(mani["layers"]))


layers: 60


In [9]:
# H2) 델타 인코딩 유틸
import numpy as np

def col_ind_to_deltas_per_row(row_ptr: np.ndarray, col_ind: np.ndarray) -> np.ndarray:
    """각 row마다 prev=0으로 시작해 col 인덱스의 델타를 기록"""
    deltas = np.empty_like(col_ind)
    w = 0
    for r in range(len(row_ptr)-1):
        s, e = int(row_ptr[r]), int(row_ptr[r+1])
        prev = 0
        for i in range(s, e):
            cur = int(col_ind[i])
            deltas[w] = cur - prev
            prev = cur
            w += 1
    return deltas

def row_ptr_deltas(row_ptr: np.ndarray) -> np.ndarray:
    """row_ptr 델타 = 각 row의 nnz (첫 항 0 제외)"""
    return np.diff(row_ptr).astype(np.int64)


In [10]:
# H3) 허프만 코어
import heapq
from collections import Counter

def huffman_code_lengths(freq: dict[int,int]) -> dict[int,int]:
    """심볼->빈도 -> 코드길이. 빈도 0 제외. 단일 심볼은 길이=1."""
    items = [(f, s) for s, f in freq.items() if f > 0]
    if not items:
        return {}
    if len(items) == 1:
        return {items[0][1]: 1}

    heap = [[f, [s]] for f, s in items]
    heapq.heapify(heap)
    lengths = {s:0 for _, s in items}
    while len(heap) > 1:
        f1, syms1 = heapq.heappop(heap)
        f2, syms2 = heapq.heappop(heap)
        for s in syms1: lengths[s] += 1
        for s in syms2: lengths[s] += 1
        heapq.heappush(heap, [f1+f2, syms1+syms2])
    return lengths

def canonical_codes_from_lengths(lengths: dict[int,int]) -> dict[int,str]:
    """심볼->길이 -> 캐논컬 허프만 코드(비트문자열)."""
    if not lengths:
        return {}
    pairs = sorted((l, s) for s, l in lengths.items())  # 길이, 심볼
    codes = {}
    code = 0
    prev_len = pairs[0][0]
    for l, s in pairs:
        if l > prev_len:
            code <<= (l - prev_len)
            prev_len = l
        codes[s] = format(code, f"0{l}b")
        code += 1
    return codes

class BitWriter:
    def __init__(self):
        self.buf = bytearray()
        self.bitbuf = 0
        self.nbits = 0
        self.total_bits = 0
    def write_bits(self, bits: str):
        self.total_bits += len(bits)
        for b in bits:
            self.bitbuf = (self.bitbuf << 1) | (1 if b == '1' else 0)
            self.nbits += 1
            if self.nbits == 8:
                self.buf.append(self.bitbuf)
                self.bitbuf = 0
                self.nbits = 0
    def finish(self) -> bytes:
        if self.nbits:
            self.bitbuf <<= (8 - self.nbits)
            self.buf.append(self.bitbuf)
            self.bitbuf = 0
            self.nbits = 0
        return bytes(self.buf)


In [11]:
# H4) 허프만 인코딩 실행 & 저장
import json, numpy as np
from collections import Counter

def load_layer_arrays(layer_name):
    csr = np.load(CSR_DIR / f"{layer_name.replace('.', '_')}.npz")
    ws  = np.load(WS_DIR  / f"{layer_name.replace('.', '_')}.npz")
    return (csr["row_ptr"].astype(np.int64),
            csr["col_ind"].astype(np.int64),
            ws["codebook"].astype(np.float32),
            ws["indices"].astype(np.int64))

summary = []
tot_idx_bits = tot_col_bits = tot_rpd_bits = 0
tot_idx_bytes = tot_col_bytes = tot_rpd_bytes = 0
tot_json_bytes = 0

for L in mani["layers"]:
    name   = L["name"]
    row_ptr, col_ind, codebook, indices = load_layer_arrays(name)

    # A) indices 스트림 허프만
    freq_idx = Counter(indices.tolist()) if indices.size else Counter()
    len_idx  = huffman_code_lengths(freq_idx)
    codes_idx= canonical_codes_from_lengths(len_idx)
    bw = BitWriter()
    for s in indices:
        bw.write_bits(codes_idx[int(s)])  # 단일심볼인 경우 코드 길이 1 할당됨
    idx_bytes = bw.finish()
    idx_bits  = bw.total_bits
    (HUF_DIR / f"{name.replace('.', '_')}_idx.bin").write_bytes(idx_bytes)

    # B) col_ind 델타(행 단위 초기화) 허프만
    deltas = col_ind_to_deltas_per_row(row_ptr, col_ind)
    freq_col = Counter(deltas.tolist()) if deltas.size else Counter()
    len_col  = huffman_code_lengths(freq_col)
    codes_col= canonical_codes_from_lengths(len_col)
    bw = BitWriter()
    for s in deltas:
        bw.write_bits(codes_col[int(s)])
    col_bytes = bw.finish()
    col_bits  = bw.total_bits
    (HUF_DIR / f"{name.replace('.', '_')}_col.bin").write_bytes(col_bytes)

    # C) row_ptr 델타(옵션)
    rpd_bits = 0
    rpd_bytes = 0
    meta_rpd = None
    if COMPRESS_ROW_PTR:
        rpd = row_ptr_deltas(row_ptr)
        freq_rpd = Counter(rpd.tolist()) if rpd.size else Counter()
        len_rpd  = huffman_code_lengths(freq_rpd)
        codes_rpd= canonical_codes_from_lengths(len_rpd)
        bw = BitWriter()
        for s in rpd:
            bw.write_bits(codes_rpd[int(s)])
        rpd_bytes = bw.finish()
        rpd_bits  = bw.total_bits
        (HUF_DIR / f"{name.replace('.', '_')}_rpd.bin").write_bytes(rpd_bytes)
        meta_rpd = {
            "stream": "rowptr_delta",
            "n_symbols": int(rpd.size),
            "code_lengths": {str(int(k)): int(v) for k, v in len_rpd.items()},
            "bit_length": int(rpd_bits)
        }

    # per-layer json (코드길이/비트길이 메타)
    layer_meta = {
        "name": name,
        "shape2d": L["shape2d"],
        "type": L["type"],
        "indices_meta": {
            "stream": "indices",
            "n_symbols": int(indices.size),
            "code_lengths": {str(int(k)): int(v) for k, v in len_idx.items()},
            "bit_length": int(idx_bits),
        },
        "col_delta_meta": {
            "stream": "col_delta",
            "n_symbols": int(deltas.size),
            "code_lengths": {str(int(k)): int(v) for k, v in len_col.items()},
            "bit_length": int(col_bits),
        },
        "rowptr_delta_meta": meta_rpd,
        "files": {
            "idx_bin": f"{name.replace('.', '_')}_idx.bin",
            "col_bin": f"{name.replace('.', '_')}_col.bin",
            "rpd_bin": f"{name.replace('.', '_')}_rpd.bin" if COMPRESS_ROW_PTR else None,
        }
    }
    jpath = HUF_DIR / f"{name.replace('.', '_')}.json"
    jtxt = json.dumps(layer_meta, indent=2)
    jpath.write_text(jtxt)
    tot_json_bytes += jpath.stat().st_size

    # 집계
    tot_idx_bits  += idx_bits
    tot_col_bits  += col_bits
    tot_idx_bytes += len(idx_bytes)
    tot_col_bytes += len(col_bytes)
    if COMPRESS_ROW_PTR:
        tot_rpd_bits  += rpd_bits
        tot_rpd_bytes += len(rpd_bytes)

    summary.append(layer_meta)

# 전체 manifest
manifest_huff = {
    "source_ws_dir": str(WS_DIR),
    "compress_row_ptr": COMPRESS_ROW_PTR,
    "layers": summary,
    "totals": {
        "idx_bits": int(tot_idx_bits),
        "col_bits": int(tot_col_bits),
        "rpd_bits": int(tot_rpd_bits) if COMPRESS_ROW_PTR else 0,
        "idx_bytes": int(tot_idx_bytes),
        "col_bytes": int(tot_col_bytes),
        "rpd_bytes": int(tot_rpd_bytes) if COMPRESS_ROW_PTR else 0,
        "json_bytes": int(tot_json_bytes)
    },
    "note": "bit_length은 패딩 제외 이론치, *_bytes는 패딩 포함 실제 파일 크기"
}
(HUF_DIR / "manifest_huff_ratio[0.9].json").write_text(json.dumps(manifest_huff, indent=2))
print("✅ Huffman encoding done:", HUF_DIR.resolve())


✅ Huffman encoding done: /home/yyb02274/yolov5/Deep_Compression_ratio[0.9]/huff_dump_ratio[0.9]


In [12]:
# H5) 허프만 디코딩 검증 (indices/col_delta/rowptr_delta)
import json, numpy as np, random

class BitReader:
    def __init__(self, data: bytes, bit_length: int):
        self.data = data; self.bit_length = bit_length; self.pos = 0
    def read_bit(self):
        if self.pos >= self.bit_length: raise EOFError
        b = self.data[self.pos // 8]
        bit = (b >> (7 - (self.pos % 8))) & 1
        self.pos += 1
        return bit

def decode_canonical(br, lengths: dict, n_symbols: int) -> np.ndarray:
    if not lengths or n_symbols == 0:
        return np.zeros((0,), dtype=np.int64)
    # 길이별 심볼 리스트 & 첫 코드
    by_len = {}
    for s, l in lengths.items():
        by_len.setdefault(l, []).append(s)
    for l in by_len: by_len[l].sort()
    first_code, code, prev_len = {}, 0, None
    for l in sorted(by_len):
        if prev_len is not None: code <<= (l - prev_len)
        first_code[l] = code
        code += len(by_len[l]); prev_len = l
    # 디코드
    out = np.empty((n_symbols,), dtype=np.int64)
    for i in range(n_symbols):
        val = 0; l = 0
        while True:
            val = (val << 1) | br.read_bit(); l += 1
            if l in first_code:
                off = val - first_code[l]
                if 0 <= off < len(by_len[l]):
                    out[i] = by_len[l][off]; break
    return out

def rebuild_col_from_deltas(row_ptr: np.ndarray, deltas: np.ndarray) -> np.ndarray:
    col = np.empty_like(deltas)
    w = 0
    for r in range(len(row_ptr)-1):
        s, e = int(row_ptr[r]), int(row_ptr[r+1])
        prev = 0
        for _ in range(s, e):
            d = int(deltas[w]); cur = prev + d
            col[w] = cur; prev = cur; w += 1
    return col

ok = True
for L in random.sample(mani["layers"], k=min(5, len(mani["layers"]))):
    name = L["name"]
    csr = np.load(CSR_DIR / f"{name.replace('.', '_')}.npz")
    row_ptr = csr["row_ptr"].astype(np.int64)
    col_ind = csr["col_ind"].astype(np.int64)

    meta = json.loads((HUF_DIR / f"{name.replace('.', '_')}.json").read_text())

    # indices
    idx_meta = meta["indices_meta"]
    idx_data = (HUF_DIR / f"{name.replace('.', '_')}_idx.bin").read_bytes()
    br = BitReader(idx_data, int(idx_meta["bit_length"]))
    idx_lengths = {int(k): int(v) for k, v in idx_meta["code_lengths"].items()}
    dec_idx = decode_canonical(br, idx_lengths, int(idx_meta["n_symbols"]))
    ws = np.load(WS_DIR / f"{name.replace('.', '_')}.npz")
    ok &= np.array_equal(dec_idx, ws["indices"].astype(np.int64))

    # col_delta
    col_meta = meta["col_delta_meta"]
    col_data = (HUF_DIR / f"{name.replace('.', '_')}_col.bin").read_bytes()
    br = BitReader(col_data, int(col_meta["bit_length"]))
    col_lengths = {int(k): int(v) for k, v in col_meta["code_lengths"].items()}
    dec_col_delta = decode_canonical(br, col_lengths, int(col_meta["n_symbols"]))
    ok &= np.array_equal(dec_col_delta, col_ind_to_deltas_per_row(row_ptr, col_ind))
    ok &= np.array_equal(rebuild_col_from_deltas(row_ptr, dec_col_delta), col_ind)

print("✅ Decode check:", "ALL OK" if ok else "MISMATCH FOUND")


✅ Decode check: ALL OK


In [13]:
# H6) 압축률 리포트 (Dense / CSR / WS / WS+Huffman[bin+json 합])
import numpy as np, json, pandas as pd
from pathlib import Path

def size_dense_bytes(shape2d):  # FP32 기준
    r, c = shape2d
    return int(r * c * 4)

def size_csr_bytes(layer):
    z = np.load(CSR_DIR / f"{layer.replace('.', '_')}.npz")
    return int(z["row_ptr"].nbytes + z["col_ind"].nbytes + z["values"].nbytes)

def size_ws_bytes(layer):
    z = np.load(WS_DIR / f"{layer.replace('.', '_')}.npz")
    return int(z["codebook"].nbytes + z["indices"].nbytes)

def size_ws_huf_bin_bytes(layer):
    # .bin 합 (json 제외)
    j = json.loads((HUF_DIR / f"{layer.replace('.', '_')}.json").read_text())
    s = 0
    s += (HUF_DIR / j["files"]["idx_bin"]).stat().st_size
    s += (HUF_DIR / j["files"]["col_bin"]).stat().st_size
    if j["files"]["rpd_bin"]:
        s += (HUF_DIR / j["files"]["rpd_bin"]).stat().st_size
    return s

def size_ws_huf_json_bytes(layer):
    return (HUF_DIR / f"{layer.replace('.', '_')}.json").stat().st_size

rows = []
for L in mani["layers"]:
    name = L["name"]
    dense_b = size_dense_bytes(tuple(L["shape2d"]))
    csr_b   = size_csr_bytes(name)
    ws_b    = size_ws_bytes(name)
    huf_bin_b  = size_ws_huf_bin_bytes(name)
    huf_json_b = size_ws_huf_json_bytes(name)
    rows.append({
        "layer": name,
        "dense_bytes": dense_b,
        "csr_bytes": csr_b,
        "ws_bytes": ws_b,
        "ws_huf_bin_json_bytes": huf_bin_b + huf_json_b,  # ★ bin+json 합만 사용
        "x_dense_to_csr": dense_b / max(1, csr_b),
        "x_dense_to_ws": dense_b / max(1, ws_b),
        "x_dense_to_ws_huf_bin_json": dense_b / max(1, (huf_bin_b + huf_json_b))  # ★ 최종 배율
    })

df = pd.DataFrame(rows)
df.to_csv("compress_report_per_layer_ratio[0.9].csv", index=False)

# Totals
dense_sum = int(df["dense_bytes"].sum())
csr_sum   = int(df["csr_bytes"].sum())
ws_sum    = int(df["ws_bytes"].sum())
huf_sum   = int(df["ws_huf_bin_json_bytes"].sum())

totals = {
    "dense_MB": dense_sum / 1e6,
    "csr_MB": csr_sum / 1e6,
    "ws_MB": ws_sum / 1e6,
    "ws_huf_bin_json_MB": huf_sum / 1e6,  # ★ 최종 모델 크기(bin+json)
    "x_dense_to_csr": dense_sum / max(1, csr_sum),
    "x_dense_to_ws": dense_sum / max(1, ws_sum),
    "x_dense_to_ws_huf_bin_json": dense_sum / max(1, huf_sum)  # ★ 최종 압축배율
}
json.dump(totals, open("compress_report_total_ratio[0.9].json", "w"), indent=2)
pd.DataFrame([totals]).to_csv("compress_report_total_ratio[0.9].csv", index=False)

# 원하는 형태로만 출력
print("== TOTAL ==")
print(f"orignal_MB: {totals['dense_MB']:.3f} MB")
print(f"purning_MB: {totals['csr_MB']:.3f} MB")
print(f"ws_MB: {totals['ws_MB']:.3f} MB")
print(f"huf_MB (final model size): {totals['ws_huf_bin_json_MB']:.3f} MB")
print(f"FINAL Huffman compression ratio: x{totals['x_dense_to_ws_huf_bin_json']:.2f}")


== TOTAL ==
orignal_MB: 28.862 MB
purning_MB: 5.855 MB
ws_MB: 0.725 MB
huf_MB (final model size): 0.957 MB
FINAL Huffman compression ratio: x30.14


In [ ]:
# H7) 결과물 ZIP (huff_dump/* + 리포트들)
import zipfile, os
with zipfile.ZipFile("huffman_results_bundle.zip_ratio[0.9]", "w", zipfile.ZIP_DEFLATED) as z:
    # encodings
    for p in HUF_DIR.glob("*"):
        z.write(p, arcname=f"huff_dump/{p.name}")
    # reports
    for name in ["compress_report_per_layer.csv", "compress_report_total.csv", "compress_report_total.json"]:
        if Path(name).exists():
            z.write(name, arcname=name)
print("✅ saved: huffman_results_bundle.zip")


✅ saved: huffman_results_bundle.zip


In [18]:
# === 경로 설정 ===
PRUNED_MODEL_PATH = "/home/yyb02274/yolov5/runs/train/retrain_pruned/weights/best.pt"
DATA_YAML         = "data/coco128.yaml"
ABS_TRAIN = "/home/yyb02274/datasets/coco128/images/train2017"
ABS_VAL   = "/home/yyb02274/datasets/coco128/images/train2017"

CSR_DIR = "csr_dump_ratio[0.9]"
WS_DIR  = "ws_dump_ratio[0.9]"
HUF_DIR = "huff_dump_ratio[0.9]"

PRUNE_TAG = "ratio[0.9]"  # ← 파일명 접미사로 사용

# YOLOv5 레포 루트로 이동(또는 sys.path 추가)
import os, sys, torch
os.chdir("/home/yyb02274/yolov5")  # ← 너의 yolov5 루트
print("CWD:", os.getcwd())

from models.common import DetectMultiBackend

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dm = DetectMultiBackend(PRUNED_MODEL_PATH, device=device, dnn=False, fuse=False)
model = dm.model.to(device).eval()
print("Loaded model:", type(model))


CWD: /home/yyb02274/yolov5
Loaded model: <class 'models.yolo.DetectionModel'>


In [17]:
import json, numpy as np
from pathlib import Path

CSR_DIR = Path(CSR_DIR); WS_DIR = Path(WS_DIR); HUF_DIR = Path(HUF_DIR)
mani = json.loads((CSR_DIR / "manifest_ratio[0.9].json").read_text())

# ----- Huffman BitReader & canonical decode -----
class BitReader:
    def __init__(self, data: bytes, bit_length: int):
        self.data = data; self.bit_length = bit_length; self.pos = 0
    def read_bit(self):
        if self.pos >= self.bit_length: raise EOFError
        b = self.data[self.pos // 8]
        bit = (b >> (7 - (self.pos % 8))) & 1
        self.pos += 1
        return bit

def decode_canonical(br, lengths: dict[int,int], n_symbols: int) -> np.ndarray:
    if not lengths or n_symbols == 0:
        return np.zeros((0,), dtype=np.int64)
    by_len = {}
    for s, l in lengths.items():
        by_len.setdefault(int(l), []).append(int(s))
    for l in by_len: by_len[l].sort()
    first_code, code, prev_len = {}, 0, None
    for l in sorted(by_len):
        if prev_len is not None: code <<= (l - prev_len)
        first_code[l] = code
        code += len(by_len[l]); prev_len = l
    out = np.empty((n_symbols,), dtype=np.int64)
    for i in range(n_symbols):
        val = 0; l = 0
        while True:
            val = (val << 1) | br.read_bit(); l += 1
            if l in first_code:
                off = val - first_code[l]
                if 0 <= off < len(by_len[l]):
                    out[i] = by_len[l][off]; break
    return out

def rebuild_col_from_deltas(row_ptr: np.ndarray, deltas: np.ndarray) -> np.ndarray:
    col = np.empty_like(deltas)
    w = 0
    for r in range(len(row_ptr)-1):
        s, e = int(row_ptr[r]), int(row_ptr[r+1])
        prev = 0
        for _ in range(s, e):
            d = int(deltas[w]); cur = prev + d
            col[w] = cur; prev = cur; w += 1
    return col

def csr_to_dense(row_ptr, col_ind, values, shape2d):
    rows, cols = shape2d
    dense = np.zeros((rows, cols), dtype=np.float32)
    for r in range(rows):
        s, e = int(row_ptr[r]), int(row_ptr[r+1])
        if e > s:
            dense[r, col_ind[s:e]] = values[s:e]
    return dense

# ----- 레이어별: .json 메타 + .bin 읽고 허프만 디코딩 → WS codebook으로 values 복원 → dense → 주입 -----
module_map = dict(model.named_modules())
mismatch = []

for L in mani["layers"]:
    name = L["name"]; rows, cols = L["shape2d"]; is_conv = (L["type"] == "conv2d")

    # per-layer meta
    meta = json.loads((HUF_DIR / f"{name.replace('.','_')}.json").read_text())

    # row_ptr: rpd 압축 했으면 복원, 아니면 CSR에서 로드
    if meta.get("rowptr_delta_meta"):
        m = meta["rowptr_delta_meta"]
        data = (HUF_DIR / f"{name.replace('.','_')}_rpd.bin").read_bytes()
        br = BitReader(data, int(m["bit_length"]))
        lengths = {int(k): int(v) for k, v in m["code_lengths"].items()}
        rpd = decode_canonical(br, lengths, int(m["n_symbols"]))
        row_ptr = np.zeros((len(rpd)+1,), dtype=np.int64); row_ptr[1:] = np.cumsum(rpd, dtype=np.int64)
    else:
        row_ptr = np.load(CSR_DIR / f"{name.replace('.','_')}.npz")["row_ptr"].astype(np.int64)

    # col_delta → col_ind
    m = meta["col_delta_meta"]
    data = (HUF_DIR / f"{name.replace('.','_')}_col.bin").read_bytes()
    br = BitReader(data, int(m["bit_length"]))
    lengths = {int(k): int(v) for k, v in m["code_lengths"].items()}
    deltas = decode_canonical(br, lengths, int(m["n_symbols"]))
    col_ind = rebuild_col_from_deltas(row_ptr, deltas)

    # indices → values via WS codebook
    m = meta["indices_meta"]
    data = (HUF_DIR / f"{name.replace('.','_')}_idx.bin").read_bytes()
    br = BitReader(data, int(m["bit_length"]))
    lengths = {int(k): int(v) for k, v in m["code_lengths"].items()}
    indices = decode_canonical(br, lengths, int(m["n_symbols"]))
    ws = np.load(WS_DIR / f"{name.replace('.','_')}.npz")
    codebook = ws["codebook"].astype(np.float32)
    values = codebook[indices] if indices.size else np.zeros((0,), dtype=np.float32)

    dense2d = csr_to_dense(row_ptr, col_ind, values, (rows, cols))
    if is_conv:
        O, I, kH, kW = L["shape"]; Wrec = dense2d.reshape(O, I, kH, kW)
    else:
        Wrec = dense2d

    mod = module_map.get(name, None)
    if (mod is None) or (not hasattr(mod, "weight")):
        mismatch.append(name); continue
    import torch
    with torch.no_grad():
        mod.weight.data = torch.from_numpy(Wrec).to(mod.weight.data.device, dtype=mod.weight.data.dtype)

print("Injected Huffman-decoded WS weights. mismatches:", len(mismatch), mismatch[:5])


Injected Huffman-decoded WS weights. mismatches: 0 []


In [19]:
from utils.general import check_yaml
try:
    from utils.dataloaders import check_dataset, create_dataloader
except:
    from utils.general import check_dataset
    from utils.dataloaders import create_dataloader

# yaml->dict + 경로 정규화
data_dict = check_dataset(check_yaml(DATA_YAML))

def norm_path(p):
    s = str(p).replace("／", "/").replace("\\", "/")
    while '//' in s:
        s = s.replace('//', '/')
    return s

for k in ("train","val","test"):
    if k in data_dict and data_dict[k]:
        data_dict[k] = norm_path(data_dict[k])

# 필요 시 절대경로 덮어쓰기(네 환경에 맞게)
data_dict["train"] = ABS_TRAIN
data_dict["val"]   = ABS_VAL

import torch
stride = int(getattr(model, 'stride', torch.tensor([32])).max())
IMGSZ, BATCH, WORKERS = 640, 16, 4

dataloader = create_dataloader(
    data_dict["val"], IMGSZ, BATCH, stride,
    single_cls=False, pad=0.5, rect=True, workers=WORKERS, prefix="val: "
)[0]
print("val images:", len(dataloader.dataset))

# 평가
from val import run as val_run
import pandas as pd

DEVICE = "0" if torch.cuda.is_available() else "cpu"
results, maps, _ = val_run(
    data=data_dict,
    dataloader=dataloader,
    imgsz=IMGSZ,
    batch_size=BATCH,
    model=model,
    device=DEVICE,
    iou_thres=0.6,
    single_cls=False,
    verbose=True,
    plots=False
)

P, R, mAP50, mAP5095, vbox, vobj, vcls = results
print(f"[HUF_{PRUNE_TAG}] P={P:.4f} R={R:.4f} mAP@0.5={mAP50:.4f} mAP@0.5:.95={mAP5095:.4f}")
print(f"[HUF_{PRUNE_TAG}] val/box={vbox:.5f} val/obj={vobj:.5f} val/cls={vcls:.5f}")

row = {
    "stage": f"pruned+WS+HUF_{PRUNE_TAG}",
    "metrics/precision": P,
    "metrics/recall": R,
    "metrics/mAP_0.5": mAP50,
    "metrics/mAP_0.5:0.95": mAP5095,
    "val/box_loss": vbox,
    "val/obj_loss": vobj,
    "val/cls_loss": vcls,
    "imgsz": IMGSZ, "batch": BATCH
}
save_name = f"huff_eval_metrics_{PRUNE_TAG}.csv"
pd.DataFrame([row]).to_csv(save_name, index=False)
print("Saved:", save_name)


val: Scanning /home/yyb02274/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


val images: 128


                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 8/8 [00:01<00:00,  7.16it/s]
                   all        128        929     0.0188     0.0217     0.0127    0.00658
                person        128        254     0.0114      0.236     0.0187    0.00504
               bicycle        128          6          0          0          0          0
                   car        128         46          0          0          0          0
            motorcycle        128          5          0          0          0          0
              airplane        128          6     0.0294      0.333     0.0402     0.0203
                   bus        128          7          0          0          0          0
                 train        128          3          0          0          0          0
                 truck        128         12          0          0          0          0
                  boat        128          6          0        

[HUF_ratio[0.9]] P=0.0188 R=0.0217 mAP@0.5=0.0127 mAP@0.5:.95=0.0066
[HUF_ratio[0.9]] val/box=0.00000 val/obj=0.00000 val/cls=0.00000
Saved: huff_eval_metrics_ratio[0.9].csv
